In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import math
import os
import sys
from absl import app
from absl import flags

import resnet
import data_eol as data_lib
import model as model_lib
import model_util as model_util

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [2]:
import numpy as np

In [3]:
tf.disable_eager_execution()
tf.config.set_soft_device_placement(True)

In [ ]:
train_batch_size = 2048
eval_batch_size = 2048
num_train_examples = 2847269
num_eval_examples = 351354

train_steps = (num_train_examples * 100 // train_batch_size + 1)
eval_steps = int(math.ceil(num_eval_examples / eval_batch_size))
epoch_steps = int(round(num_train_examples / train_batch_size))

labels_path = '/home/psd2120/research/data/labels.txt'
checkpoint_path = 'gs://eol-tfrc-tpu/chkpts/eol2020/finetune/experiment/ResNet50_2048/'
tpu_name = 'grpc://10.196.234.178:8470'
model_dir = 'gs://eol-tfrc-tpu/chkpts/eol2020/finetune/experiment/out/'

In [ ]:
def my_mean_acc_class(labels, predictions):
    return {'mean_acc_class': tf.metrics.mean_per_class_accuracy(labels, predictions, 72626)}

In [ ]:
eval_steps = int(math.ceil(num_eval_examples / eval_batch_size))

with tf.io.gfile.GFile(labels_path, 'r') as f:
    x = f.read().splitlines()
num_classes = len(x)

In [ ]:
resnet.BATCH_NORM_DECAY = 0.9
model = resnet.resnet_v1(resnet_depth=50,
                         width_multiplier=1,
                         cifar_stem=False)

In [ ]:
reader = tf.train.load_checkpoint(tf.train.latest_checkpoint(checkpoint_path))

In [ ]:
for x in reader:
    print(x)

In [ ]:
cluster = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_name)
default_eval_mode = tf.estimator.tpu.InputPipelineConfig.PER_HOST_V1
sliced_eval_mode = tf.estimator.tpu.InputPipelineConfig.SLICED

model_fn = model_lib.build_model_fn(model, num_classes, num_train_examples)
input_fn = data_lib.build_input_fn(num_classes, False)
tpu_config = tf.estimator.tpu.TPUConfig(iterations_per_loop=train_steps,
                                        eval_training_input_configuration=sliced_eval_mode)

In [ ]:
run_config = tf.estimator.tpu.RunConfig(
    tpu_config=tpu_config,
    model_dir=model_dir,
    save_summary_steps=100,
    save_checkpoints_steps=epoch_steps,
    keep_checkpoint_max=5,
    master=None,
    cluster=cluster)

In [ ]:
estimator = tf.estimator.tpu.TPUEstimator(model_fn,
                                          config=run_config,
                                          train_batch_size=train_batch_size,
                                          eval_batch_size=eval_batch_size,
                                          use_tpu=True)

In [ ]:
est = tf.estimator.Estimator(model_fn, checkpoint_path)

In [ ]:
input_fn = data_lib.build_input_fn(num_classes, False)
# estimator = tf.estimator.add_metrics(estimator, my_mean_acc_class)

In [ ]:
result = est.evaluate(input_fn, eval_steps, checkpoint_path=checkpoint_path, name='sanity')

In [ ]:
tf.estimator.ModeKeys.TRAIN

In [4]:
preds = tf.placeholder(tf.int64, [1,8])
labels = tf.Variable([1, 1, 2, 2, 2, 3, 3, 0])

acc, acc_op = tf.metrics.mean_per_class_accuracy(labels, preds, 5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
sess = tf.Session()

sess.run(tf.local_variables_initializer())
sess.run(tf.global_variables_initializer())

stream_vars = [i for i in tf.local_variables()]
print(stream_vars)

[<tf.Variable 'mean_accuracy/total:0' shape=(5,) dtype=float32>, <tf.Variable 'mean_accuracy/count:0' shape=(5,) dtype=float32>]


In [6]:
print('acc:',sess.run(acc, {preds:np.array([1, 4, 2, 1, 3, 3, 3, 0]).reshape(1, 8)}))
#acc: 0.0

print('[total, count]:',sess.run(stream_vars)) 
#[total, count]: [0.0, 0.0]

acc: 0.0
[total, count]: [array([0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0.], dtype=float32)]


In [7]:
print('ops:', sess.run(acc_op, {preds:np.array([1, 4, 2, 1, 3, 3, 3, 0]).reshape(1, 8)})) 
#ops: 1.0

print('[total, count]:',sess.run(stream_vars)) 
#[total, count]: [2.0, 2.0]

ops: [1.         0.5        0.33333334 1.         0.        ]
[total, count]: [array([1., 2., 3., 2., 0.], dtype=float32), array([1., 1., 1., 2., 0.], dtype=float32)]


In [8]:
print('acc:', sess.run(acc, {preds:np.array([1, 4, 2, 1, 3, 3, 3, 0]).reshape(1, 8)}))

acc: 0.5666667
